# 1. Install and Import Libraries

In [ ]:
import os
import json
import random
from copy import deepcopy
from pathlib import Path
import time
from google.colab import drive

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BertForSequenceClassification,
    AlbertForSequenceClassification,
    BertTokenizer,
    get_linear_schedule_with_warmup,
)
from torch.optim import AdamW

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

In [ ]:
import random

In [ ]:
!pip install optuna

In [ ]:
import optuna

# 2. Environment Settings and Initialization

## 2.1. Set Seed for Reproducibility

In [ ]:
SEED = 42

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def seed_worker(worker_id):
    worker_seed = SEED + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)


def make_generator(seed):
    g = torch.Generator()
    g.manual_seed(seed)
    return g

## 2.2. Set GPU

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

## 2.3. Manage Google Drive

### 2.3.1. Mount Google Drive

In [ ]:
drive.mount('/content/drive')

### 2.3.2. Set Folder Path

In [ ]:
# Datasets
READ_PATH = '/content/drive/MyDrive/Bach_Thesis/Dataset/'

# Models
STUDY_ROOT = "/content/drive/MyDrive/Bach_Thesis/Models/SA_Optuna"
SAVE_ROOT = "/content/drive/MyDrive/Bach_Thesis/Models/SA_Check"

## 2.4. Initialize Global Parameters

In [ ]:
LABELS = [0,1]
TARGET_NAMES = ['negative' ,'positive']
NUM_LABELS = 2

## 2.5. Initialize Datasets

In [ ]:
def extract_xy(df, text_col="cleaned_content", label_col="sentiment"):
    X = df[text_col].tolist()
    y = df[label_col].tolist()
    return X, y

In [ ]:
df_train_ori = pd.read_csv(f"{READ_PATH}Train2lab.csv")
df_train_ros = pd.read_csv(f"{READ_PATH}Train_ROS2lab.csv")
df_train_ros_ncl = pd.read_csv(f"{READ_PATH}Train_ROS_NCL2lab.csv")
df_val = pd.read_csv(f"{READ_PATH}Validation2lab.csv")
df_test = pd.read_csv(f"{READ_PATH}Test2lab.csv")

In [ ]:
X_train, y_train = extract_xy(df_train_ori)
X_train_ros, y_train_ros = extract_xy(df_train_ros)
X_train_ros_ncl, y_train_ros_ncl = extract_xy(df_train_ros_ncl)
X_val, y_val     = extract_xy(df_val)
X_test, y_test   = extract_xy(df_test)

In [ ]:
class_weights = torch.tensor(
    compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train),
    dtype=torch.float
).to(DEVICE)

class_weights_ros = torch.tensor(
    compute_class_weight(class_weight='balanced', classes=np.unique(y_train_ros), y=y_train_ros),
    dtype=torch.float
).to(DEVICE)

class_weights_ros_ncl = torch.tensor(
    compute_class_weight(class_weight='balanced', classes=np.unique(y_train_ros_ncl), y=y_train_ros_ncl),
    dtype=torch.float
).to(DEVICE)

# 3. Helper Functions

In [ ]:
MAX_LEN = int(df_train_ori['cleaned_content'].str.split().str.len().max())
MAX_LEN

## 3.1. Dataset Class

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = list(texts)
        self.labels = list(labels)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

## 3.2. Initialization Model and Tokenizer

In [ ]:
def init_model_and_tokenizer(MODEL_NAME, num_labels=NUM_LABELS, hidden_dropout_prob=None, attention_dropout_prob=None):
    low = MODEL_NAME.lower()
    model_kwargs = {}

    if hidden_dropout_prob is not None:
        model_kwargs['hidden_dropout_prob'] = float(hidden_dropout_prob)
    if attention_dropout_prob is not None:
        model_kwargs['attention_probs_dropout_prob'] = float(attention_dropout_prob)

    if "lite" in low:
        model = AlbertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels, **model_kwargs)
        tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
    elif "nusabert" in low:
        model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels, **model_kwargs)
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
    else:
        model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels, **model_kwargs)
        tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

    return model, tokenizer

## 3.3. Create Classification Report

In [ ]:
def create_classification_report(y_true, y_pred):
    cr = classification_report(
        y_true, y_pred,
        labels=LABELS,
        target_names=TARGET_NAMES,
        zero_division=0,
        output_dict=True
    )
    df_cr = pd.DataFrame(cr).transpose().reset_index()
    df_cr = df_cr.rename(columns={'index': 'label'})

    for col in df_cr.select_dtypes(include=['float']).columns:
        df_cr[col] = df_cr[col].apply(lambda x: round(x, 4))

    return df_cr

## 3.4. Create Confusion Matrix

In [ ]:
def create_confusion_matrix(y_true, y_pred, save_path_png):
    cm = confusion_matrix(y_true, y_pred, labels=LABELS)
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=TARGET_NAMES, yticklabels=TARGET_NAMES)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig(save_path_png, dpi=150)
    plt.close()

## 3.5. Full Pipeline (Hyperparameter Tuning -> Final Model Training -> Final Model Testing)

In [ ]:
def get_best_hyperparameter(save_root, model_name, dataset_name):
    model_dir = model_name.replace("/", "_")
    experiment_dir = Path(save_root) / model_dir / dataset_name
    optuna_dir = experiment_dir / "optuna_study"
    storage_path = optuna_dir / "study.db"
    study_name_path = optuna_dir / "study_name.txt"
    storage_uri = f"sqlite:///{storage_path}"

    if storage_path.exists() and study_name_path.exists():
        study_name = study_name_path.read_text().strip()

        try:
            loaded_study = optuna.load_study(
                study_name=study_name,
                storage=storage_uri
            )

            best_trial = loaded_study.best_trial
            return best_trial.params

        except Exception as e:
            print(f"An error occurred while loading study: {e}")
            return None
    else:
        print(f"Error: Study files not found at expected path: {optuna_dir}")
        return None

In [ ]:
def get_hp_grid(best_hp, change_hp=None):
    if change_hp is None:
        return best_hp

    grid = best_hp.copy()
    for key, value in change_hp.items():
        grid[key] = value
    return grid

In [ ]:
def train_validate_model(model, train_loader, val_loader, optimizer, scheduler, loss_fn, device):
    # TRAIN
    model.train()
    total_train_loss = 0.0

    for batch in tqdm(train_loader, desc="Training", leave=False):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = loss_fn(outputs.logits, labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_loader)

    # VAL
    model.eval()
    total_val_loss = 0.0
    val_preds, val_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            loss = loss_fn(outputs.logits, labels)
            total_val_loss += loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)
    val_f1 = f1_score(val_labels, val_preds, average="weighted")

    return val_f1, avg_train_loss, avg_val_loss

In [ ]:
def pipeline(model_name, X_train, y_train, X_val, y_val, X_test, y_test, dataset_name, hyperparams, class_weights, device=DEVICE, save_root=SAVE_ROOT
):-
    seed_everything(SEED)

    print("\nRunning experiment with hyperparameters:")
    print(hyperparams)

    model, tokenizer = init_model_and_tokenizer(
        model_name,
        num_labels=NUM_LABELS,
        hidden_dropout_prob=hyperparams["hidden_dropout_prob"],
        attention_dropout_prob=hyperparams["attention_dropout_prob"]
    )
    model.to(device)

    train_loader = DataLoader(
        SentimentDataset(X_train, y_train, tokenizer, MAX_LEN),
        batch_size=hyperparams["batch_size"],
        shuffle=True,
        collate_fn=collate_fn,
        worker_init_fn=seed_worker,
        generator=make_generator(SEED),
        num_workers=0
    )

    val_loader = DataLoader(
        SentimentDataset(X_val, y_val, tokenizer, MAX_LEN),
        batch_size=hyperparams["batch_size"],
        shuffle=False,
        collate_fn=collate_fn,
        worker_init_fn=seed_worker,
        generator=make_generator(SEED),
        num_workers=0
    )

    test_loader = DataLoader(
        SentimentDataset(X_test, y_test, tokenizer, MAX_LEN),
        batch_size=hyperparams["batch_size"],
        shuffle=False,
        collate_fn=collate_fn,
        worker_init_fn=seed_worker,
        generator=make_generator(SEED),
        num_workers=0
    )

    loss_fn = nn.CrossEntropyLoss(weight=class_weights)

    optimizer = AdamW(
        model.parameters(),
        lr=hyperparams["lr"],
        weight_decay=hyperparams["weight_decay"]
    )

    total_steps = len(train_loader) * hyperparams["epochs"]
    warmup_steps = int(total_steps * hyperparams["warmup_ratio"])

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    history = []

    for epoch in range(hyperparams["epochs"]):
        val_f1, train_loss, val_loss = train_validate_model(
            model,
            train_loader,
            val_loader,
            optimizer,
            scheduler,
            loss_fn,
            device
        )

        history.append({
            "epoch": epoch + 1,
            "train_loss": float(train_loss),
            "val_loss": float(val_loss),
            "val_weighted_f1": float(val_f1)
        })

    history_df = pd.DataFrame(history)

    def evaluate(loader):
        preds, trues = [], []
        model.eval()
        with torch.no_grad():
            for batch in loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels_batch = batch['labels'].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
                trues.extend(labels_batch.cpu().numpy())
        return preds, trues

    train_preds, train_true = evaluate(train_loader)
    val_preds, val_true = evaluate(val_loader)
    test_preds, test_true = evaluate(test_loader)

    train_cr = create_classification_report(train_true, train_preds)
    val_cr = create_classification_report(val_true, val_preds)
    test_cr = create_classification_report(test_true, test_preds)

    train_cr["dataset"] = "train"
    val_cr["dataset"] = "validation"
    test_cr["dataset"] = "test"

    combined = pd.concat([train_cr, val_cr, test_cr], ignore_index=True)

    return combined, history_df, model, tokenizer, {
        "train": (train_true, train_preds),
        "val": (val_true, val_preds),
        "test": (test_true, test_preds)
    }

In [ ]:
def check_loop(model_name, dataset_name, best_hp, hp_change_list, class_weights, X_train, y_train, X_val, y_val, X_test, y_test):

    results = []

    for i, change_hp in enumerate(hp_change_list, start=1):
        seed_everything(SEED)

        print(f"GENERATING MODEL {i}")
        print(f"Override: {change_hp}")

        final_hp = get_hp_grid(best_hp, change_hp)
        print(f"Final hyperparameters used:")
        print(final_hp)

        cr, history_df, model, tokenizer, preds_dict = pipeline(
            model_name=model_name,
            X_train=X_train, y_train=y_train,
            X_val=X_val, y_val=y_val,
            X_test=X_test, y_test=y_test,
            dataset_name=dataset_name,
            hyperparams=final_hp,
            class_weights=class_weights
        )

        model_dir = model_name.replace("/", "_")
        base_path = Path(SAVE_ROOT) / model_dir / dataset_name / f"Run_{i}"

        cr_dir = base_path / "cr"
        model_save_dir = base_path / "model"
        cm_dir = base_path / "confusion_matrix"
        history_dir = base_path / "history"

        for d in [cr_dir, model_save_dir, cm_dir, history_dir]:
            d.mkdir(parents=True, exist_ok=True)

        history_path = history_dir / "history.csv"
        history_df.to_csv(history_path, index=False)
        print(f"Saved history: {history_path}")

        cr_path = cr_dir / "classification_report.csv"
        cr.to_csv(cr_path, index=False)
        print(f"Saved CR: {cr_path}")

        model.save_pretrained(model_save_dir)
        tokenizer.save_pretrained(model_save_dir)
        print(f"Saved model to {model_save_dir}")

        create_confusion_matrix(preds_dict["train"][0], preds_dict["train"][1], cm_dir / "train_cm.png")
        create_confusion_matrix(preds_dict["val"][0], preds_dict["val"][1], cm_dir / "val_cm.png")
        create_confusion_matrix(preds_dict["test"][0], preds_dict["test"][1], cm_dir / "test_cm.png")
        print(f"Saved confusion matrices to {cm_dir}")

        results.append({
            "run_id": i,
            "changed_hyperparams": change_hp,
            "final_hyperparams": final_hp,
            "cr_path": str(cr_path),
            "model_path": str(model_save_dir)
        })

        print("="*40)
        print()

    return pd.DataFrame(results)

# INDOBERT-LITE-BASE-P1

In [ ]:
mod_indo_lite_p1 = "indobenchmark/indobert-lite-base-p1"

## original

In [ ]:
best_hp_indo_lite_ori_p1 = get_best_hyperparameter(STUDY_ROOT, mod_indo_lite_p1, "original")

In [ ]:
list_lite_p1_ori = [
    {'hidden_dropout_prob': 0.1},
    {'hidden_dropout_prob': 0.2},
    {'attention_dropout_prob': 0.1},
    {'attention_dropout_prob': 0.25},
    {'lr': 1e-6},
    {'lr': 1e-4},
    {'warmup_ratio': 0.01},
    {'warmup_ratio': 0.1}
]

check_loop(
    model_name=mod_indo_lite_p1,
    dataset_name="ori-check-1",
    best_hp=best_hp_indo_lite_ori_p1,
    hp_change_list=list_lite_p1_ori,
    class_weights=class_weights,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
list_lite_p1_ori = [
    {'attention_dropout_prob': 0.1, 'warmup_ratio': 0.01}
]

check_loop(
    model_name=mod_indo_lite_p1,
    dataset_name="ori-check-2",
    best_hp=best_hp_indo_lite_ori_p1,
    hp_change_list=list_lite_p1_ori,
    class_weights=class_weights,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
list_lite_p1_ori = [
    {'attention_dropout_prob': 0.13},
    {'attention_dropout_prob': 0.05},
    {'attention_dropout_prob': 0.01},
    {'attention_dropout_prob': 0.005},
    {'hidden_dropout_prob': 0.13},
    {'hidden_dropout_prob': 0.05},
    {'hidden_dropout_prob': 0.01},
    {'hidden_dropout_prob': 0.005},
]

check_loop(
    model_name=mod_indo_lite_p1,
    dataset_name="ori-check-3",
    best_hp=best_hp_indo_lite_ori_p1,
    hp_change_list=list_lite_p1_ori,
    class_weights=class_weights,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
list_lite_p1_ori = [
    {'warmup_ratio': 0.02},
    {'warmup_ratio': 0.005},
    {'warmup_ratio': 0.001}
]

check_loop(
    model_name=mod_indo_lite_p1,
    dataset_name="ori-check-4",
    best_hp=best_hp_indo_lite_ori_p1,
    hp_change_list=list_lite_p1_ori,
    class_weights=class_weights,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

## ros

In [ ]:
best_hp_indo_lite_ros_p1 = get_best_hyperparameter(STUDY_ROOT, mod_indo_lite_p1, "ROS")

list_lite_p1_ros = [
   {'hidden_dropout_prob': 0.05},
   {'hidden_dropout_prob': 0.2},
   {'warmup_ratio': 0.001},
   {'warmup_ratio': 0.1}
]

check_loop(
    model_name=mod_indo_lite_p1,
    dataset_name="ros-check-1",
    best_hp=best_hp_indo_lite_ros_p1,
    hp_change_list=list_lite_p1_ros,
    class_weights=class_weights_ros,
    X_train=X_train_ros, y_train=y_train_ros,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
best_hp_indo_lite_ros_p1 = get_best_hyperparameter(STUDY_ROOT, mod_indo_lite_p1, "ROS")

list_lite_p1_ros = [
   {'warmup_ratio': 0.15},
   {'warmup_ratio': 0.2},
   {'warmup_ratio': 0.25},
   {'warmup_ratio': 0.3},
]

check_loop(
    model_name=mod_indo_lite_p1,
    dataset_name="ros-check-2",
    best_hp=best_hp_indo_lite_ros_p1,
    hp_change_list=list_lite_p1_ros,
    class_weights=class_weights_ros,
    X_train=X_train_ros, y_train=y_train_ros,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

## ros-ncl

In [ ]:
best_hp_indo_lite_ros_ncl_p1 = get_best_hyperparameter(STUDY_ROOT, mod_indo_lite_p1, "ROS-NCL")

list_lite_p1_ros_ncl = [
   {'hidden_dropout_prob': 0.01},
   {'hidden_dropout_prob': 0.2},
   {'warmup_ratio': 0.01},
   {'warmup_ratio': 0.1},
   {'batch_size': 16},
   {'batch_size': 64},
   {'attention_dropout_prob': 0.01},
   {'attention_dropout_prob': 0.4}
]

check_loop(
    model_name=mod_indo_lite_p1,
    dataset_name="rosncl-check-1",
    best_hp=best_hp_indo_lite_ros_ncl_p1,
    hp_change_list=list_lite_p1_ros_ncl,
    class_weights=class_weights_ros_ncl,
    X_train=X_train_ros_ncl, y_train=y_train_ros_ncl,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
best_hp_indo_lite_ros_ncl_p1 = get_best_hyperparameter(STUDY_ROOT, mod_indo_lite_p1, "ROS-NCL")

list_lite_p1_ros_ncl = [
   {'warmup_ratio': 0.15},
   {'warmup_ratio': 0.2},
   {'warmup_ratio': 0.25},
   {'warmup_ratio': 0.3},
]

check_loop(
    model_name=mod_indo_lite_p1,
    dataset_name="rosncl-check-2",
    best_hp=best_hp_indo_lite_ros_ncl_p1,
    hp_change_list=list_lite_p1_ros_ncl,
    class_weights=class_weights_ros_ncl,
    X_train=X_train_ros_ncl, y_train=y_train_ros_ncl,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

# INDOBERT-BASE-P1

In [ ]:
mod_indo_base_p1 = "indobenchmark/indobert-base-p1"

## original

In [ ]:
best_hp_indo_base_ori_p1 = get_best_hyperparameter(STUDY_ROOT, mod_indo_base_p1, "original")

In [ ]:
list_base_p1_ori = [
   {'epochs': 10},
   {'epochs': 5},
   {'lr': 1e-06},
   {'lr': 0.0001},
   {'hidden_dropout_prob': 0.3},
   {'hidden_dropout_prob': 0.15},
   {'batch_size': 32},
   {'batch_size': 128}
]

check_loop(
    model_name=mod_indo_base_p1,
    dataset_name="ori-check-1",
    best_hp=best_hp_indo_base_ori_p1,
    hp_change_list=list_base_p1_ori,
    class_weights=class_weights,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

## ros

In [ ]:
best_hp_indo_base_ros_p1 = get_best_hyperparameter(STUDY_ROOT, mod_indo_base_p1, "ROS")

list_base_p1_ros = [
   {'warmup_ratio': 0.001},
   {'warmup_ratio': 0.1},
   {'epochs': 2},
   {'epochs': 5},
   {'lr': 1e-06},
   {'lr': 0.0001},
   {'hidden_dropout_prob': 0.35},
   {'hidden_dropout_prob': 0.2},
   {'attention_dropout_prob': 0.1},
   {'attention_dropout_prob': 0.2}
]

check_loop(
    model_name=mod_indo_base_p1,
    dataset_name="ros-check-1",
    best_hp=best_hp_indo_base_ros_p1,
    hp_change_list=list_base_p1_ros,
    class_weights=class_weights_ros,
    X_train=X_train_ros, y_train=y_train_ros,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

## ros-ncl


In [ ]:
best_hp_indo_base_ros_ncl_p1 = get_best_hyperparameter(STUDY_ROOT, mod_indo_base_p1, "ROS-NCL")

list_base_p1_ros_ncl = [
   {'lr': 1e-06},
   {'lr': 0.0001},
   {'batch_size': 128},
   {'batch_size': 32},
   {'hidden_dropout_prob': 0.45},
   {'hidden_dropout_prob': 0.3},
   {'attention_dropout_prob': 0.4},
   {'attention_dropout_prob': 0.2}
]

check_loop(
    model_name=mod_indo_base_p1,
    dataset_name="rosncl-check-1",
    best_hp=best_hp_indo_base_ros_ncl_p1,
    hp_change_list=list_base_p1_ros_ncl,
    class_weights=class_weights_ros_ncl,
    X_train=X_train_ros_ncl, y_train=y_train_ros_ncl,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
best_hp_indo_base_ros_ncl_p1 = get_best_hyperparameter(STUDY_ROOT, mod_indo_base_p1, "ROS-NCL")

list_base_p1_ros_ncl = [
   {'batch_size': 128, 'hidden_dropout_prob': 0.45},
   {'batch_size': 32, 'hidden_dropout_prob': 0.45},
   {'batch_size': 128, 'hidden_dropout_prob': 0.3},
   {'batch_size': 32, 'hidden_dropout_prob': 0.3}
]

check_loop(
    model_name=mod_indo_base_p1,
    dataset_name="rosncl-check-2",
    best_hp=best_hp_indo_base_ros_ncl_p1,
    hp_change_list=list_base_p1_ros_ncl,
    class_weights=class_weights_ros_ncl,
    X_train=X_train_ros_ncl, y_train=y_train_ros_ncl,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
best_hp_indo_base_ros_ncl_p1 = get_best_hyperparameter(STUDY_ROOT, mod_indo_base_p1, "ROS-NCL")

list_base_p1_ros_ncl = [
   {'batch_size': 256},
   {'hidden_dropout_prob': 0.25},
   {'hidden_dropout_prob': 0.2},
   {'hidden_dropout_prob': 0.15},
   {'hidden_dropout_prob': 0.1},
]

check_loop(
    model_name=mod_indo_base_p1,
    dataset_name="rosncl-check-3",
    best_hp=best_hp_indo_base_ros_ncl_p1,
    hp_change_list=list_base_p1_ros_ncl,
    class_weights=class_weights_ros_ncl,
    X_train=X_train_ros_ncl, y_train=y_train_ros_ncl,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

# INDOBERT-LITE-BASE-P2

In [ ]:
mod_indo_lite_p2 = "indobenchmark/indobert-lite-base-p2"

## original

In [ ]:
best_hp_indo_lite_ori_p2 = get_best_hyperparameter(STUDY_ROOT, mod_indo_lite_p2, "original")

list_lite_p2_ori = [
   {'batch_size': 16},
   {'batch_size': 64},
   {'hidden_dropout_prob': 0.01},
   {'hidden_dropout_prob': 0.2},
   {'warmup_ratio': 0.01},
   {'warmup_ratio': 0.1}
]

check_loop(
    model_name=mod_indo_lite_p2,
    dataset_name="ori-check-1",
    best_hp=best_hp_indo_lite_ori_p2,
    hp_change_list=list_lite_p2_ori,
    class_weights=class_weights,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
best_hp_indo_lite_ori_p2 = get_best_hyperparameter(STUDY_ROOT, mod_indo_lite_p2, "original")

list_lite_p2_ori = [
   {'batch_size': 16, 'hidden_dropout_prob': 0.2},
   {'batch_size': 64, 'hidden_dropout_prob': 0.2}
]

check_loop(
    model_name=mod_indo_lite_p2,
    dataset_name="ori-check-2",
    best_hp=best_hp_indo_lite_ori_p2,
    hp_change_list=list_lite_p2_ori,
    class_weights=class_weights,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
best_hp_indo_lite_ori_p2 = get_best_hyperparameter(STUDY_ROOT, mod_indo_lite_p2, "original")

list_lite_p2_ori = [
   {'hidden_dropout_prob': 0.15},
   {'hidden_dropout_prob': 0.25},
   {'hidden_dropout_prob': 0.3},
   {'hidden_dropout_prob': 0.35},
   {'hidden_dropout_prob': 0.4},
]

check_loop(
    model_name=mod_indo_lite_p2,
    dataset_name="ori-check-3",
    best_hp=best_hp_indo_lite_ori_p2,
    hp_change_list=list_lite_p2_ori,
    class_weights=class_weights,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

## ros

In [ ]:
best_hp_indo_lite_ros_p2 = get_best_hyperparameter(STUDY_ROOT, mod_indo_lite_p2, "ROS")

list_lite_p2_ros = [
   {'warmup_ratio': 0.1},
   {'warmup_ratio': 0.2},
   {'batch_size': 16},
   {'batch_size': 32},
   {'hidden_dropout_prob': 0.01},
   {'hidden_dropout_prob': 0.2},
   {'attention_dropout_prob': 0.1},
   {'attention_dropout_prob': 0.2},
   {'weight_decay': 1e-06},
   {'weight_decay': 0.0001}
]

check_loop(
    model_name=mod_indo_lite_p2,
    dataset_name="ros-check-1",
    best_hp=best_hp_indo_lite_ros_p2,
    hp_change_list=list_lite_p2_ros,
    class_weights=class_weights_ros,
    X_train=X_train_ros, y_train=y_train_ros,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
best_hp_indo_lite_ros_p2 = get_best_hyperparameter(STUDY_ROOT, mod_indo_lite_p2, "ROS")

list_lite_p2_ros = [
   {'batch_size': 64, 'weight_decay': 1e-06},
   {'batch_size': 64, 'weight_decay': 0.0001},
   {'batch_size': 64}
]

check_loop(
    model_name=mod_indo_lite_p2,
    dataset_name="ros-check-2",
    best_hp=best_hp_indo_lite_ros_p2,
    hp_change_list=list_lite_p2_ros,
    class_weights=class_weights_ros,
    X_train=X_train_ros, y_train=y_train_ros,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
best_hp_indo_lite_ros_p2 = get_best_hyperparameter(STUDY_ROOT, mod_indo_lite_p2, "ROS")

list_lite_p2_ros = [
   {'weight_decay': 1e-6},
   {'weight_decay': 1e-5},
   {'weight_decay': 1e-4}
]

check_loop(
    model_name=mod_indo_lite_p2,
    dataset_name="ros-check-3",
    best_hp=best_hp_indo_lite_ros_p2,
    hp_change_list=list_lite_p2_ros,
    class_weights=class_weights_ros,
    X_train=X_train_ros, y_train=y_train_ros,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
best_hp_indo_lite_ros_p2 = get_best_hyperparameter(STUDY_ROOT, mod_indo_lite_p2, "ROS")

list_lite_p2_ros = [
   {'weight_decay': 1e-7},
   {'weight_decay': 1e-8},
   {'weight_decay': 1e-9}
]

check_loop(
    model_name=mod_indo_lite_p2,
    dataset_name="ros-check-4",
    best_hp=best_hp_indo_lite_ros_p2,
    hp_change_list=list_lite_p2_ros,
    class_weights=class_weights_ros,
    X_train=X_train_ros, y_train=y_train_ros,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

## ros-ncl

In [ ]:
best_hp_indo_lite_ros_ncl_p2 = get_best_hyperparameter(STUDY_ROOT, mod_indo_lite_p2, "ROS-NCL")

list_lite_p2_ros_ncl = [
   {'hidden_dropout_prob': 0.1},
   {'hidden_dropout_prob': 0.2},
   {'warmup_ratio': 0.01},
   {'warmup_ratio': 0.1},
   {'batch_size': 16},
   {'batch_size': 32}
]

check_loop(
    model_name=mod_indo_lite_p2,
    dataset_name="rosncl-check-1",
    best_hp=best_hp_indo_lite_ros_ncl_p2,
    hp_change_list=list_lite_p2_ros_ncl,
    class_weights=class_weights_ros_ncl,
    X_train=X_train_ros_ncl, y_train=y_train_ros_ncl,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
best_hp_indo_lite_ros_ncl_p2 = get_best_hyperparameter(STUDY_ROOT, mod_indo_lite_p2, "ROS-NCL")

list_lite_p2_ros_ncl = [
   {'batch_size': 64}
]

check_loop(
    model_name=mod_indo_lite_p2,
    dataset_name="rosncl-check-2",
    best_hp=best_hp_indo_lite_ros_ncl_p2,
    hp_change_list=list_lite_p2_ros_ncl,
    class_weights=class_weights_ros_ncl,
    X_train=X_train_ros_ncl, y_train=y_train_ros_ncl,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

# INDOBERT-BASE-P2

In [ ]:
mod_indo_base_p2 = "indobenchmark/indobert-base-p2"

## original

In [ ]:
best_hp_indo_base_ori_p2 = get_best_hyperparameter(STUDY_ROOT, mod_indo_base_p2, "original")

In [ ]:
list_base_p2_ori = [
   {'attention_dropout_prob': 0.4},
   {'attention_dropout_prob': 0.2},
   {'warmup_ratio': 0.25},
   {'warmup_ratio': 0.1},
   {'batch_size': 128},
   {'batch_size': 32},
   {'hidden_dropout_prob': 0.45},
   {'hidden_dropout_prob': 0.3},
   {'lr': 1e-06},
   {'lr': 0.0001}
]

check_loop(
    model_name=mod_indo_base_p2,
    dataset_name="ori-check-1",
    best_hp=best_hp_indo_base_ori_p2,
    hp_change_list=list_base_p2_ori,
    class_weights=class_weights,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
list_base_p2_ori = [
   {'hidden_dropout_prob': 0.3},
   {'hidden_dropout_prob': 0.25},
   {'hidden_dropout_prob': 0.2},
   {'hidden_dropout_prob': 0.15},
]

check_loop(
    model_name=mod_indo_base_p2,
    dataset_name="ori-check-2",
    best_hp=best_hp_indo_base_ori_p2,
    hp_change_list=list_base_p2_ori,
    class_weights=class_weights,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

## ros

In [ ]:
best_hp_indo_base_ros_p2 = get_best_hyperparameter(STUDY_ROOT, mod_indo_base_p2, "ROS")

list_base_p2_ros = [
   {'warmup_ratio': 0.0001},
   {'warmup_ratio': 0.01},
   {'lr': 1e-06},
   {'lr': 0.0001}
]

check_loop(
    model_name=mod_indo_base_p2,
    dataset_name="ros-check-1",
    best_hp=best_hp_indo_base_ros_p2,
    hp_change_list=list_base_p2_ros,
    class_weights=class_weights_ros,
    X_train=X_train_ros, y_train=y_train_ros,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

## ros-ncl

In [ ]:
best_hp_indo_base_ros_ncl_p2 = get_best_hyperparameter(STUDY_ROOT, mod_indo_base_p2, "ROS-NCL")

list_base_p2_ros_ncl = [
   {'hidden_dropout_prob': 0.45},
   {'hidden_dropout_prob': 0.3},
   {'attention_dropout_prob': 0.45},
   {'attention_dropout_prob': 0.4}
]

check_loop(
    model_name=mod_indo_base_p2,
    dataset_name="rosncl-check-1",
    best_hp=best_hp_indo_base_ros_ncl_p2,
    hp_change_list=list_base_p2_ros_ncl,
    class_weights=class_weights_ros_ncl,
    X_train=X_train_ros_ncl, y_train=y_train_ros_ncl,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
best_hp_indo_base_ros_ncl_p2 = get_best_hyperparameter(STUDY_ROOT, mod_indo_base_p2, "ROS-NCL")

list_base_p2_ros_ncl = [
   {'hidden_dropout_prob': 0.45, 'attention_dropout_prob': 0.45}
]

check_loop(
    model_name=mod_indo_base_p2,
    dataset_name="rosncl-check-2",
    best_hp=best_hp_indo_base_ros_ncl_p2,
    hp_change_list=list_base_p2_ros_ncl,
    class_weights=class_weights_ros_ncl,
    X_train=X_train_ros_ncl, y_train=y_train_ros_ncl,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
best_hp_indo_base_ros_ncl_p2 = get_best_hyperparameter(STUDY_ROOT, mod_indo_base_p2, "ROS-NCL")

list_base_p2_ros_ncl = [
   {'attention_dropout_prob': 0.3},
   {'attention_dropout_prob': 0.45},
   {'attention_dropout_prob': 0.5},
   {'hidden_dropout_prob': 0.3},
   {'hidden_dropout_prob': 0.45},
   {'hidden_dropout_prob': 0.5},
]

check_loop(
    model_name=mod_indo_base_p2,
    dataset_name="rosncl-check-3",
    best_hp=best_hp_indo_base_ros_ncl_p2,
    hp_change_list=list_base_p2_ros_ncl,
    class_weights=class_weights_ros_ncl,
    X_train=X_train_ros_ncl, y_train=y_train_ros_ncl,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
best_hp_indo_base_ros_ncl_p2 = get_best_hyperparameter(STUDY_ROOT, mod_indo_base_p2, "ROS-NCL")

list_base_p2_ros_ncl = [
   {'attention_dropout_prob': 0.5, 'hidden_dropout_prob': 0.45}
]

check_loop(
    model_name=mod_indo_base_p2,
    dataset_name="rosncl-check-4",
    best_hp=best_hp_indo_base_ros_ncl_p2,
    hp_change_list=list_base_p2_ros_ncl,
    class_weights=class_weights_ros_ncl,
    X_train=X_train_ros_ncl, y_train=y_train_ros_ncl,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

# NUSABERT-BASE

In [ ]:
mod_nusa = "LazarusNLP/NusaBERT-base"

## original

In [ ]:
best_hp_nusa_ori = get_best_hyperparameter(STUDY_ROOT, mod_nusa, "original")

In [ ]:
list_nusa_ori = [
   {'warmup_ratio': 0.1},
   {'warmup_ratio': 0.2},
   {'lr': 1e-06},
   {'lr': 0.0001},
   {'hidden_dropout_prob': 0.1},
   {'hidden_dropout_prob': 0.25}
]

check_loop(
    model_name=mod_nusa,
    dataset_name="ori-check-1",
    best_hp=best_hp_nusa_ori,
    hp_change_list=list_nusa_ori,
    class_weights=class_weights,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

## ros

In [ ]:
best_hp_nusa_ros = get_best_hyperparameter(STUDY_ROOT, mod_nusa, "ROS")

list_nusa_ros = [
   {'attention_dropout_prob': 0.3},
   {'attention_dropout_prob': 0.1},
   {'batch_size': 32},
   {'batch_size': 128},
   {'hidden_dropout_prob': 0.1},
   {'hidden_dropout_prob': 0.2},
   {'warmup_ratio': 0.001},
   {'warmup_ratio': 0.01}
]

check_loop(
    model_name=mod_nusa,
    dataset_name="ros-check-1",
    best_hp=best_hp_nusa_ros,
    hp_change_list=list_nusa_ros,
    class_weights=class_weights_ros,
    X_train=X_train_ros, y_train=y_train_ros,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

## ros-ncl

In [ ]:
best_hp_nusa_ros_ncl = get_best_hyperparameter(STUDY_ROOT, mod_nusa, "ROS-NCL")

list_nusa_ros_ncl = [
   {'hidden_dropout_prob': 0.05},
   {'hidden_dropout_prob': 0.2},
   {'warmup_ratio': 0.01},
   {'warmup_ratio': 0.1}
]

check_loop(
    model_name=mod_nusa,
    dataset_name="rosncl-check-1",
    best_hp=best_hp_nusa_ros_ncl,
    hp_change_list=list_nusa_ros_ncl,
    class_weights=class_weights_ros_ncl,
    X_train=X_train_ros_ncl, y_train=y_train_ros_ncl,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)

In [ ]:
best_hp_nusa_ros_ncl = get_best_hyperparameter(STUDY_ROOT, mod_nusa, "ROS-NCL")

list_nusa_ros_ncl = [
   {'hidden_dropout_prob': 0.18},
   {'hidden_dropout_prob': 0.25},
   {'hidden_dropout_prob': 0.3},
   {'hidden_dropout_prob': 0.35},
   {'hidden_dropout_prob': 0.4}
]

check_loop(
    model_name=mod_nusa,
    dataset_name="rosncl-check-2",
    best_hp=best_hp_nusa_ros_ncl,
    hp_change_list=list_nusa_ros_ncl,
    class_weights=class_weights_ros_ncl,
    X_train=X_train_ros_ncl, y_train=y_train_ros_ncl,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test
)